<a href="https://colab.research.google.com/github/bidhyabhattarai/Abstractive-Summarization-/blob/main/Abstractive_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers


In [2]:
!pip install transformers gradio


In [1]:
!pip install gradio


In [14]:
!pip install contractions


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.9 MB/s eta 0:00:00


In [20]:
!pip install transformers gradio beautifulsoup4 nltk contractions


In [28]:
# Importing Required Libraries
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import gradio as gr
import re

# Load the Pegasus Model and Tokenizer
model_name = "google/pegasus-xsum"  # Change to "google/pegasus-cnn_dailymail" for longer summaries
print(f"Loading model: {model_name}")
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Preprocessing Function
def preprocess_text(text):
    """
    Preprocesses the input text by removing extra spaces, special characters, and ensuring clean text.
    """
    # Remove special characters except for periods, commas, and question marks
    text = re.sub(r'[^a-zA-Z0-9\s.,?]', '', text)

    # Remove extra spaces
    text = " ".join(text.split())

    return text

# Define the Summarization Function
def summarize_text(text, max_length=200, min_length=50, length_penalty=1.0, num_beams=8):
    """
    Summarizes the input text using the Pegasus model and ensures the summary starts with a capital letter.
    """
    if len(text.strip()) == 0:
        return "⚠️ Please enter some text to summarize!"

    # Preprocess the input text
    text = preprocess_text(text)

    # Tokenize input text
    input_ids = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)

    # Generate Summary
    summary_ids = model.generate(
        input_ids,
        max_length=max_length,  # Maximum length of the summary
        min_length=min_length,
        length_penalty=length_penalty,
        num_beams=num_beams,
        early_stopping=True,
    )

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Postprocess the summary: Capitalize the first letter and strip extra spaces
    summary = summary.strip()  # Remove any leading or trailing spaces
    if summary:
        summary = summary[0].upper() + summary[1:]

    return summary

# Build the Gradio Interface
description = """
<div style="text-align:center;">
    <h2>🔍 Abstractive Text Summarizer</h2>
    <p>Leverage the power of Pegasus to generate concise and meaningful summaries.</p>
    <p><i>Adjust the parameters to customize the summarization as per your requirements!</i></p>
</div>
"""

# Gradio Interface
interface = gr.Interface(
    fn=summarize_text,
    inputs=[
        gr.Textbox(
            lines=10,
            label="Input Text",
            placeholder="Type or paste your text here to summarize...",
        ),
        gr.Slider(
            label="Maximum Summary Length", minimum=50, maximum=500, step=10, value=100
        ),
        gr.Slider(
            label="Minimum Summary Length", minimum=100, maximum=250, step=5, value=20
        ),
        gr.Slider(
            label="Length Penalty", minimum=1.0, maximum=2.0, step=0.1, value=1.5
        ),
        gr.Slider(
            label="Number of Beams (Quality)", minimum=2, maximum=8, step=1, value=4
        ),
    ],
    outputs=gr.Textbox(label="Generated Summary"),
    title="📝 Abstractive Text Summarization with Pegasus",
    description=description,
    theme="compact",  # Light and clean theme
    allow_flagging="never",  # Disable flagging for a cleaner interface
)

# Launch the Gradio App
interface.launch()


Loading model: google/pegasus-xsum


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1048: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-6783cbcf-4474b8155a8aaa4f7426b57c;398dd0ad-0804-4081-9671-f423f8761767)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d03d49a69a23c587cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
